In [ ]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install checktoxicity #our own library
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install gradio -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.2

In [ ]:
from peft import *
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from toxic.toxic import Toxic
import gradio as gr

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    load_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
peft_model_id = "nebulae7/eight"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(model, peft_model_id)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 100
generation_config.temperature = 0.5
generation_config.top_p = 0.9
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 11,
  "max_new_tokens": 200,
  "pad_token_id": 11,
  "temperature": 0.5,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

In [ ]:
DEVICE = "cuda:0"

In [ ]:
import re
def last_moderate(sentence):
  keyword_counts = {'can i': 0,'should i': 0,'should we': 0,'recommend': 0,'advice': 0,'invest': 0,'investment': 0,'stock': 0,'stocks': 0,'mutual funds': 0,'mutual fund': 0,'how': 0,'what': 0,'when': 0,'where': 0}
  for keyword in keyword_counts:
    keyword_counts[keyword] = sentence.count(keyword)
    for count in keyword_counts.values():
      if count >= 2:
        return True
      return False

In [ ]:
import re
investment_keywords = ['invest', 'investment', 'stock', 'stocks', 'mutual funds', 'mutual fund']
question_keywords = ['how', 'what', 'when', 'where']
investment_sentiments = ['can i','should i', 'should we', 'recommend', 'advice']
def contains_investment_advice(sentence):
    sentence_lower = sentence.lower()
    has_investment_keyword = any(keyword in sentence_lower for keyword in investment_keywords)
    has_question_keyword = any(keyword in sentence_lower for keyword in question_keywords)
    has_investment_sentiment = any(sentiment in sentence_lower for sentiment in investment_sentiments)
    print(has_investment_keyword and has_question_keyword or has_investment_keyword and has_investment_sentiment or has_question_keyword and has_investment_sentiment)
    if has_investment_keyword and has_question_keyword or has_investment_keyword and has_investment_sentiment or has_question_keyword and has_investment_sentiment:
      return True
    return last_moderate(sentence_lower)

In [ ]:
def remove_repetitive_sentences(output_text):
    sentences = output_text.strip().split('. ')
    unique_sentences = []
    for sentence in sentences:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    cleaned_output = '. '.join(unique_sentences)
    return cleaned_output

In [ ]:
def moderate_layer(response):
  result = Toxic(response,"hf_QohiChEs******") #my token_ID
  return remove_repetitive_sentences(result.getScoreFromAPI())

In [ ]:
def generate_response(question: str) -> str:
    prompt = f"""
    <human>: {question}
    <assistant>:
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True,clean_up_tokenization_space=True)
    assistant_start = '<assistant>:'
    assistant_stop = '.'
    response_start = response.find(assistant_start)
    response_stop = response.find(assistant_stop)
    return moderate_layer(response[response_start + len(assistant_start):].strip().split("\n")[0])

In [ ]:
# import os
# from reportlab.lib.pagesizes import letter
# from reportlab.pdfgen import canvas

In [ ]:
moderate_response_text = "As an AI model, I cannot provide specific financial advice. Please consult a financial professional for personalized guidance."

In [ ]:
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import simpleSplit
import gradio as gr
def chat():
    with gr.Blocks() as demo:
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Input")
        clear = gr.ClearButton([msg, chatbot])

        def respond(message, chat_history):
            global bot_message
            if contains_investment_advice(message):
                bot_message = moderate_response_text
            else:
                bot_message = generate_response(message)
            chat_history.append((message, bot_message))
            return "", chat_history
        msg.submit(respond, [msg, chatbot], [msg, chatbot])

        gr.Examples(
            [
                ["How do I login into my account at Fidelity International?"],
                ["What is Vanguard Life Strategy 100?"],
                ["What is the percentage increase in average hourly earnings at this finance company over the past 12 months?"],
                ["What is the trend of the participation rate for Jp Morgan over the years, including the recent increase to 75%?"],
                ["What is the minimum investment required with AJ Bell?"],
                ["How can Interactive Investor help me with tax planning strategy?"],
                ["What were the fluctuations in the 2-year and 10-year Treasury yield from January 2023 to June 2023, including the rise and fall of interest rates?"],
            ],
            inputs=msg,
            label="Click on any example and press Enter in the input textbox!",
        )

    return demo
def pdf():
  with gr.Blocks() as demo:
      iface = gr.Interface(
        fn=generate_pdf,
        inputs=[],
        outputs="file",
        live=True,
        capture_session=True,
        description="Click the button to generate and download the PDF file.",
        allow_flagging=False
      )
      return iface

# def generate_pdf():
#     filename = "generated_pdf.pdf"
#     c = canvas.Canvas(filename, pagesize=letter)
#     x = 72
#     y = 720
#     text_width = 400

#     c.setFont("Helvetica", 12)

#     # Split the text into lines using word-wrapping
#     lines = simpleSplit(generate_braille_pdf(bot_message), "Helvetica", 12, text_width)

#     for line in lines:
#         c.drawString(x, y, line)
#         # Move to the next line by adjusting the y-coordinate
#         y -= 20  # Adjust the value to set the spacing between lines

#     c.save()
#     return filename

def get_demo():
    with gr.Blocks(css="""#chatbot {font-size: 14px; min-height: 300px;}""") as demo:
      chat()
      #pdf()
    return demo
demo = get_demo()
demo.launch()


<ipython-input-17-343a5f07c1c7>:39: GradioDeprecationWarning: `capture_session` parameter is deprecated, and it has no effect
  iface = gr.Interface(
/usr/local/lib/python3.10/dist-packages/gradio/interface.py:331: UserWarning: The `allow_flagging` parameter in `Interface` nowtakes a string value ('auto', 'manual', or 'never'), not a boolean. Setting parameter to: 'never'.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# generate_response("How do I login into my account at Fidelity International?")

In [ ]:
# # Braille mapping dictionary
# from google.colab import files
# import pdfkit

# braille_mapping = {
#     'a': '⠁', 'b': '⠃', 'c': '⠉', 'd': '⠙', 'e': '⠑', 'f': '⠋', 'g': '⠛',
#     'h': '⠓', 'i': '⠊', 'j': '⠚', 'k': '⠅', 'l': '⠇', 'm': '⠍', 'n': '⠝',
#     'o': '⠕', 'p': '⠏', 'q': '⠟', 'r': '⠗', 's': '⠎', 't': '⠞', 'u': '⠥',
#     'v': '⠧', 'w': '⠺', 'x': '⠭', 'y': '⠽', 'z': '⠵',
#     ' ': ' ', '\n': '\n', '\r': ''
# }

# def char_to_braille(char):
#     return braille_mapping.get(char.lower(), char)

# def text_to_braille(text):
#     return ''.join(char_to_braille(char) for char in text)

# def generate_braille_pdf(text):
#   braille_text = text_to_braille(text)
#   return braille_text

In [ ]:
# def generate_pdf(text):
#   filename = "generated_pdf.pdf"
#   c = canvas.Canvas(filename, pagesize=letter)
#   c.drawString(72, 720, text)
#   c.save()
#   return filename